Профессия Data Science  
Блок 2. Подгрузка данных  
**SQL-5. Типы данных**

---

## **✍ Оглавление:**

2. Типы данных в PostgreSQL
3. Даты: основные типы
4. Функции и операторы для работы с датами

---

## **2. Типы данных в PostgreSQL**

**ОБЗОР ТИПОВ ДАННЫХ**

Особенностью хранения данных в БД является их строгая **типизация**, то есть точное и явное определение типов. Необходимость в типизации обусловлена тем, что компьютер по-разному обрабатывает даты, целые или дробные числа, строки.

Поэтому при создании таблицы БД обязательно указывают типы данных, которые будут сохраняться в каждом из её полей, — от этого зависит, какие значения допустимы в этих полях.

Типы данных — это один из основных видов ограничений в PostgreSQL.

**ОСНОВНЫЕ ТИПЫ ДАННЫХ В POSTGRESQL**

**SQL** — это язык со строгой типизацией, в котором каждый элемент данных имеет некоторый тип, определяющий его поведение и допустимое использование.

Типы данных в PostgreSQL можно разделить на несколько групп. К основным относятся:

- числовые типы — для хранения чисел (целых и дробных);  
- типы даты/времени — для хранения даты, времени, часовых поясов;  
- символьные типы — для хранения символов или строк;  
- логический тип — для хранения значений типа «истина», «ложь».

**В зависимости от требований к хранимой информации необходимо правильно применять типы данных. На это есть как минимум две причины.**

1. Разные типы данных могут занимать разный **объём памяти**.

2. На преобразование типов данных тратится **время**.

## **3. Даты: основные типы**

→ Любой анализ событий во времени подразумевает работу с датой или временем.

Для них в Postgres существует несколько типов данных.

**TIMESTAMP**

**Timestamp** — наиболее распространённый тип данных, так как он содержит и дату, и время, а также используется в любых логах событий, временных рядах и в большинстве системных таблиц.

Согласно стандарту ISO, значение выглядит как "2019-07-14 01:35:44.702165+00", где перечислены через точку год-месяц-день, время и часовой пояс.

Для получения текущего значения даты и времени в Postgres используются функции **CURRENT_TIMESTAMP** (есть в стандарте SQL) и **NOW()** (есть в большинстве баз данных).

**SELECT NOW() и SELECT CURRENT_TIMESTAMP**

**TIMESTAMP WITH TIME ZONE**

**Timestamp with time zone** позволяет хранить сведения о часовом поясе, что может быть удобно при анализе географически распределённых временных данных для единообразия хранения.

Сначала попробуем узнать, в каком часовом поясе выводятся временные данные в настоящий момент. Для этого выполните команду

**show timezone**

В результате вы увидите GMT — это наиболее частая установка по умолчанию для баз данных. 

Список часовых поясов можно увидеть в системном справочнике pg_timezone_names.

***select now() at time zone 'Europe/Moscow'***

Указание at time zone позволяет переводить дату/время без часового пояса в дату/время с часовым поясом и обратно, а также пересчитывать значения времени для различных часовых поясов.

**Задание 3.1 (External resource)**

select now() at time zone 'America/Los_Angeles' as now

**DATE**

С типом date вы уже знакомы, его реализация предельно проста. Отметим только, что тип timestamp (with/without time zone) можно легко перевести в соответствующую дату, используя синтаксис

***"timestamp_column"::date***

И наоборот, тип date преобразуется в timestamp (дата и 00:00:00) с помощью

***"date_column"::timestamp***

Для получения текущей даты можно использовать

***select CURRENT_DATE***

или

***select now()::date***

**Задание 3.2 (External resource)**

WITH x AS   
(SELECT '2018-12-31 21:00:00+00'::timestamp WITH time zone ts)  
SELECT (ts at time zone 'Europe/Moscow')::date dt_msk, (ts)::date dt_utc  
FROM x

**INTERVAL**

**Interval** — тип данных, позволяющий хранить разницу между двумя временными метками. 

Интервалы хранят данные в трёх отдельных полях — месяцах, днях, секундах. Это сделано из-за того, что количество дней в месяце и часов в дне может быть разным. Пример значения такого типа: "195 days -10:52:23.563955".

## **4. Функции и операторы для работы с датами**

#### **ФУНКЦИЯ EXTRACT()** - получает из значений даты/времени такие поля, как год или час.

Здесь источник — значение типа timestamp, time или interval. Допускается и тип date, поскольку он приводится к типу timestamp.

Указанное поле представляет собой идентификатор, по которому из источника выбирается заданное поле. Функция extract() возвращает значения типа double precision.

**! DAY**

Для значений timestamp это день месяца (1-31), для значений interval — число дней.

***SELECT EXTRACT(DAY FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 16

***SELECT EXTRACT(DAY FROM INTERVAL '40 days 1 minute');***  
Результат: 40

**! HOUR**

Час (0-23).

***SELECT EXTRACT(HOUR FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 20

**! MONTH**

Номер месяца, считая с января (1) до декабря (12).

***SELECT EXTRACT(MONTH FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 2

**! YEAR**

Поле года. Учтите, что года 0 не было, и это следует иметь в виду, вычитая из годов нашей эры годы до нашей эры.

***SELECT EXTRACT(YEAR FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 2001

**! ISOYEAR**

Год по недельному календарю ISO 8601, в который попадает дата (не применимо к интервалам).

***SELECT EXTRACT(ISOYEAR FROM DATE '2006-01-01');***  
Результат: 2005

***SELECT EXTRACT(ISOYEAR FROM DATE '2006-01-02');***  
Результат: 2006

Год по недельному календарю ISO начинается с понедельника недели, в которой оказывается 4 января, так что в начале января или в конце декабря год по ISO может отличаться от года по григорианскому календарю. Подробнее об этом рассказывается в описании поля week.

**! WEEK**

Номер недели в году по недельному календарю ISO 8601. По определению, недели ISO 8601 начинаются с понедельника, а первая неделя года включает 4 января этого года. Другими словами, первый четверг года всегда оказывается в первой неделе этого года.

В системе нумерации недель ISO первые числа января могут относиться к 52-й или 53-й неделе предыдущего года, а последние числа декабря — к первой неделе следующего года.

Например, 2005-01-01 относится к 53-й неделе 2004 г., а 2006-01-01 — к 52-й неделе 2005 г., тогда как 2012-12-31 включается в первую неделю 2013 г.

Поэтому для получения согласованных результатов рекомендуется использовать поле isoyear в паре с week.

***SELECT EXTRACT(WEEK FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 7

**! CENTURY**

***SELECT EXTRACT(CENTURY FROM TIMESTAMP '2000-12-16 12:21:13');***  
Результат: 20

***SELECT EXTRACT(CENTURY FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 21

Первый век начался 0001-01-01 00:00:00, хотя люди в то время так и не считали. Это определение распространяется на все страны с григорианским календарём.

Века с номером 0 не было; считается, что 1 наступил после -1.

**! DECADE** Десятилетие.

***SELECT EXTRACT(DECADE FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 200

**! EPOCH**

Для значений timestamp with time zone это число секунд с 1970-01-01 00:00:00 UTC (может быть отрицательным); для значений date и timestamp это число секунд с 1970-01-01 00:00:00 по местному времени, а для interval — общая длительность интервала в секундах.

***SELECT EXTRACT(EPOCH FROM TIMESTAMP WITH TIME ZONE '2001-02-16 20:38:40.12-08');***  
Результат: 982384720.12

***SELECT EXTRACT(EPOCH FROM INTERVAL '5 days 3 hours');***  
Результат: 442800

Преобразовать время эпохи обратно, в значение дата/время, с помощью to_timestamp можно так:

***SELECT to_timestamp(982384720.12);***  
Результат: 2001-02-17 04:38:40.12+00

**! DOW** День недели, считая с воскресенья (0) до субботы (6).

***SELECT EXTRACT(DOW FROM TIMESTAMP '2001-02-16 20:38:40');***  
Результат: 5

Заметьте, что в extract() дни недели нумеруются не так, как в функции to_char(..., 'D').

**! DOY** День года (1-365/366).

***SELECT EXTRACT(DOY FROM TIMESTAMP '2001-02-16 20:38:40');***   
Результат: 47

**! ISODOW** День недели, считая с понедельника (1) до воскресенья (7).

***SELECT EXTRACT(ISODOW FROM TIMESTAMP '2001-02-18 20:38:40');***  
Результат: 7

Результат отличается от dow только для воскресенья. Такая нумерация соответствует ISO 8601.

**Задание 4.1 (External resource)**

#### **ФУНКЦИЯ TO_CHAR()** - нужна для форматирования даты времени и интервалов в нужный текст.

По результату работы она очень близка к extract(), но больше нацелена именно на форматирование. 

to_char(timestamp[date],text) - преобразует время в текст  
to_char(current_timestamp, 'HH12:MI:SS')

to_char(interval, text) - преобразует интервал в текст  
to_char(interval '15h 2m 12s', 'HH24:MI:SS')

Первый параметр для функции — дата, время или интервал, а второй — текстовая маска получаемого результата. 

***select to_char(now(),'"Hello! Today is" DAY yyyy-Mon-dd')***

**Задание 4.2 (External resource)**

#### **ФУНКЦИЯ DATE_TRUNC()** - позволяет отсечь заданное время, дату или дату со временем до нужной точности.

Формат вызова:

**date_trunc('поле', значение)**

Например, если мы хотим округлить текущее время-дату до минут, то можно вызвать

***select date_trunc('minute',now())***

Для получения разной степени точности вместо minute можно использовать следующие параметры: microseconds; milliseconds; second; minute; hour; day; week; month; quarter; year; decade; century; millennium.

**Задание 4.3 (External resource)**

#### **МАТЕМАТИЧЕСКИЕ ОПЕРАТОРЫ**

К любой дате можно прибавить (и вычесть из неё) целое число X и получить другую дату, которая больше (меньше) изначальной.

Пример:

***select '2019-01-01'::date + 10***  
Результат: '2019-01-11'

Это — дата на 10 дней позже 2019-01-01.

При добавлении (или вычитании) целого числа к дате Postgres учитывает переходы между месяцами и годами и даёт верный ответ, соответствующий календарю. Учитываются даже високосные годы.

Пример:

***select '2019-01-01'::date + 500***  
Результат: '2020-05-15'

Как видим, сменились и год, и месяц, и день.

Аналогично можно вычесть из одной даты другую и получить расстояние в днях между этими датами. При такой операции тоже будет честная разница по календарю.

Пример:

***select '2019-02-10'::date - '2017-03-01'::date***  
Результат: 711

**Задание 4.4 (External resource)**